In [3]:
! pip install pydot 
! pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 737.7 kB/s eta 0:00:00a 0:00:01


In [1]:
import math, pydot, graphviz
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pydot'

In [ ]:
#!pip install pydot

In [ ]:
#!pip install graphviz

In [ ]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [ ]:
df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")

In [ ]:
df.head()

In [ ]:
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])

In [ ]:
#split to train andtest while keeping each user in the train and test data
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)

In [ ]:
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]

## Implemation of TenserFlow

In [ ]:
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [ ]:
# user pipeline
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users)+1, output_dim=32)(user_as_integer)

In [ ]:
# book pipeline
book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books)+1, output_dim=32)(book_as_integer)

In [ ]:
# dot product
dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
flatten = tf.keras.layers.Flatten()(dot)

In [ ]:
# model input/output definition
model = tf.keras.Model(inputs=[user_input, book_input], outputs=flatten)

In [ ]:
model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
## this model is quite bad with an avg of 3 starts in rating deviation
# model.fit(
#     x={
#         "user": x_train["user_id"],
#         "book": x_train["isbn_num"]
#     },
#     y=y_train.values,
#     batch_size=256,
#     epochs=100,
#     validation_split=0.1, # for early stopping
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
#     ],
# )


# full pipline

In [ ]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 4 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [ ]:
#  model.fit(
#      x={
#          "user": x_train["user_id"],
#          "book": x_train["isbn_num"]
#      },
#      y=y_train.values,
#      batch_size=256,
#      epochs=4,
#      validation_split=0.1, # for early stopping
#      callbacks=[
#          tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
#      ],
#  )


add user data

In [ ]:
user_input = tf.keras.layers.Input(shape=(2,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 4 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [ ]:
 model.fit(
     x={
         "user": x_train[["user_id",'age','']],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=256,
     epochs=5,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
     ],
 )


In [ ]:
pred = model.predict(x_test)

In [ ]:
pred = model.predict({
    "user": x_test[["user_id",'age','Language','country']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
})


In [ ]:
model.evaluate({
    "user": x_test[["user_id",'age','Language','country']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
},y_test)

In [ ]:
y_test

In [ ]:
pred

In [ ]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils import model_to_dot
keras.utils.pydot = pydot
plot_model(model)